In [1]:
def get_historical_data(pair, max_requests):
    
    db_update_status = db_update_pair_table(pair,'building')

    num_requests = 0
    
    for i in range(10):
        
        url = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime=1241893500000'
        
        try:

            oldest_klines = requests.get(url)
            
            num_requests+= 1
            
            break
            
        except Exception as error:
            
            print('Comunicação com a binance falhou na entrada, erro: ',error) #Na verdade enviar para destino apropriado

            time.sleep(5)
        
    if (oldest_klines):
        
        db_save_kline_status = db_save_kline(pair,oldest_klines)
        
        if (db_save_kline_status == 'done'):
            
            step = 60000
            delta_time = 500*step
            start_time = int((oldest_klines.json()[0][0])) + delta_time
            ciclo = 0
            
            while True:
                
                for i in range(10):
                    
                    url = 'https://api.binance.com/api/v1/klines?symbol='+pair+'&interval=1m&startTime='+str(start_time)
                    
                    try:
                        
                        new_klines = requests.get(url)
                        
                        num_requests+= 1
                        
                        break

                    except Exception as error:
                        
                        print('Comunicação com a binance falhou no cliclo ',ciclo,', tentativa ',i,'. Erro: ', error) #Na verdade enviar para destino apropriado

                        time.sleep(5)

                if (new_klines):
                    
                    if (len(new_klines.json()) < 500): #Chegou no mais recente
                        
                        print('dado mais atual, este foi o último ciclo')
                        
                        new_klines = new_klines[:(len(new_klines)-1)]
                        
                    db_save_kline_status = db_save_kline(pair,new_klines)
                    
                    start_time = start_time + delta_time
                            
                    if (len(new_klines.json()) < 500): 

                        break

                    if (len(new_klines.json()) > 500): #Alguma anomalia.

                        print('ATENÇÃO, MAIS DE 500 AMOSTRAS')
                        break

                print ('Número de requests: ', num_requests)

                if(num_requests > int(max_requests)):

                    print('número máximo de requests atingido, dormindo')
                    num_requests = 0
                    time.sleep(60)

                ciclo+=1

            #Atualiza, na tabela "getdata_pairs", os 'historical' dos pares "absent", de "building" para "full"
            db_update_status = db_update_pair_table(pair,'full')
    
    return klines

In [ ]:
def db_save_kline(pair,binance_pair_klines):
    
    